# Object Detection with PyTorch and Mask R-CNN 

In this tutorial, you will finetune a pre-trained [Mask R-CNN](https://arxiv.org/abs/1703.06870) model on images from the [Penn-Fudan Database for Pedestrian Detection and Segmentation](https://www.cis.upenn.edu/~jshi/ped_html/). The dataset has 170 images with 345 instances of pedestrians.

## Prerequisities

- If you are using an Azure Machine Learning Notebook VM, your environment already meets these prerequisites. Otherwise, go through the [Configuration](https://docs.microsoft.com/azure/machine-learning/how-to-configure-environment) steps to install the Azure Machine Learning Python SDK and [create an Azure ML Workspace](https://docs.microsoft.com/azure/machine-learning/how-to-manage-workspace#create-a-workspace).


In [4]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)


SDK version: 1.0.65


## Diagnostics

Opt-in diagnostics for better experience, quality, and security in future releases.

In [5]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


## Initialize a workspace

Initialize a [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`, using the [from_config()](https://docs.microsoft.com/python/api/azureml-core/azureml.core.workspace(class)?view=azure-ml-py#from-config-path-none--auth-none---logger-none---file-name-none-) method.

In [4]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace name: gopalv-ws
Azure region: westus2
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: aifxdemo


## Create or attach existing Azure ML Managed Compute

You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/concept-compute-target) for training your model. In this tutorial, we use [Azure ML managed compute](https://docs.microsoft.com/azure/machine-learning/how-to-set-up-training-targets#amlcompute) for our remote training compute resource. Specifically, the below code creates a `STANDARD_NC6` GPU cluster that autoscales from 0 to 4 nodes.

**Creation of Compute takes approximately 5 minutes.** If the Aauzre ML Compute with that name is already in your workspace, this code will skip the creation process. 

As with other Azure servies, there are limits on certain resources associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/how-to-manage-quotas) on the default limits and how to request more quota.

> Note that the below code creates GPU compute. If you instead want to create CPU compute, provide a different VM size to the `vm_size` parameter, such as `STANDARD_D2_V2`.

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# choose a name for your cluster
cluster_name = 'gpu-cluster'

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-03-06T20:09:23.778000+00:00', 'errors': None, 'creationTime': '2020-03-05T14:55:28.441267+00:00', 'modifiedTime': '2020-03-05T15:05:30.544256+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


## Train model on the remote compute

### Create a project directory
Create a directory that will contain all the code from your local machine that you will need access to on the remote resource. This includes the training script an any additional files your training script depends on.

In [8]:
import os

project_folder = './pytorch-peds'

try:
    os.makedirs(project_folder, exist_ok=False)
except FileExistsError:
    print(f'project folder {project_folder} exists, moving on...')

project folder ./pytorch-peds exists, moving on...


Possibly helpful: [this link](https://github.com/drabastomek/GTC/blob/master/SJ_2020/workshop/1_Setup/Setup.ipynb), and this sample dockerfile from Jordan:

```
FROM mcr.microsoft.com/azureml/base-gpu:intelmpi2018.3-cuda9.0-cudnn7-ubuntu16.04

# Install Horovod, temporarily using CUDA stubsddd 
RUN ldconfig /usr/local/cuda/lib64/stubs && \     
# Install AzureML SDK     
pip install --no-cache-dir azureml-defaults && \     
# Install PyTorch     
pip install --no-cache-dir tensorflow==2.0.0b1 tensorflow-gpu==2.0.0b1 keras==2.0.8 matplotlib==3.0.3 seaborn==0.9.0 requests==2.21.0 bs4==0.0.1 imageio==2.5.0 sklearn pandas==0.24.2 numpy==1.16.2 hickle==3.4.3 && \     
# Install Horovod     
pip install --no-cache-dir horovod==0.13.5 && \     ldconfig
```

### Copy training script and dependencies into project directory

In [9]:
import shutil

shutil.copy('data.py', project_folder)
shutil.copy('model.py', project_folder)
shutil.copy('script.py', project_folder)

files_to_copy = ['utils', 'transforms', 'coco_eval', 'engine', 'coco_utils']
for file in files_to_copy:
    shutil.copy('./'+ file + '.py', project_folder)

In [ ]:
# !git clone https://github.com/pytorch/vision.git

# !git checkout v0.3.0

# %cd vision
# !cp references/detection/utils.py ../
# !cp references/detection/transforms.py ../
# !cp references/detection/coco_eval.py ../
# !cp references/detection/engine.py ../
# !cp references/detection/coco_utils.py ../
# %cd ..



## Prepare Data

### Download data locally

First we download the sample dataset, and extract the images into local storage.

In [22]:
import os
import urllib.request

from zipfile import ZipFile

data_file = 'PennFudanPed.zip'
ds_path = 'PennFudanPedData'

os.makedirs(name=ds_path, exist_ok=True)

urllib.request.urlretrieve('https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip', data_file)
zip = ZipFile(file=data_file)
zip.extractall(path=ds_path)

### Upload the data to Azure blob storage

Next, we upload the data files to the datastore associated with this workspace, so that we can access them during training.

In [23]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

ds.upload(src_dir=os.path.join(ds_path, zip.namelist()[0]).rstrip('/'), target_path=ds_path, overwrite=False)
#ds.upload(src_dir='PennFudanData', target_path=ds_path, overwrite=False)

workspaceblobstore AzureBlob gopalvws3790775563 azureml-blobstore-e47496c6-9688-4277-a05b-ceb722514b9d
Uploading an estimated of 512 files
Uploading PennFudanPedData/PennFudanPed/added-object-list.txt
Uploading PennFudanPedData/PennFudanPed/readme.txt
Uploading PennFudanPedData/PennFudanPed/Annotation/FudanPed00001.txt
Uploading PennFudanPedData/PennFudanPed/Annotation/FudanPed00002.txt
Uploading PennFudanPedData/PennFudanPed/Annotation/FudanPed00003.txt
Uploading PennFudanPedData/PennFudanPed/Annotation/FudanPed00004.txt
Uploading PennFudanPedData/PennFudanPed/Annotation/FudanPed00005.txt
Uploading PennFudanPedData/PennFudanPed/Annotation/FudanPed00006.txt
Uploading PennFudanPedData/PennFudanPed/Annotation/FudanPed00007.txt
Uploading PennFudanPedData/PennFudanPed/Annotation/FudanPed00008.txt
Uploading PennFudanPedData/PennFudanPed/Annotation/FudanPed00009.txt
Uploading PennFudanPedData/PennFudanPed/Annotation/FudanPed00010.txt
Uploading PennFudanPedData/PennFudanPed/Annotation/FudanPe

Uploaded PennFudanPedData/PennFudanPed/Annotation/FudanPed00063.txt, 43 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/Annotation/FudanPed00073.txt
Uploaded PennFudanPedData/PennFudanPed/Annotation/FudanPed00065.txt, 44 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/Annotation/FudanPed00074.txt
Uploaded PennFudanPedData/PennFudanPed/Annotation/FudanPed00066.txt, 45 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/Annotation/PennPed00001.txt
Uploaded PennFudanPedData/PennFudanPed/Annotation/FudanPed00070.txt, 46 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/Annotation/PennPed00002.txt
Uploaded PennFudanPedData/PennFudanPed/Annotation/FudanPed00068.txt, 47 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/Annotation/PennPed00003.txt
Uploaded PennFudanPedData/PennFudanPed/Annotation/FudanPed00069.txt, 48 files out of an estimated total

Uploaded PennFudanPedData/PennFudanPed/Annotation/PennPed00041.txt, 109 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/Annotation/PennPed00064.txt
Uploaded PennFudanPedData/PennFudanPed/Annotation/FudanPed00022.txt, 110 files out of an estimated total of 512
Uploaded PennFudanPedData/PennFudanPed/Annotation/PennPed00044.txt, 111 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/Annotation/PennPed00065.txt
Uploaded PennFudanPedData/PennFudanPed/Annotation/PennPed00045.txt, 112 files out of an estimated total of 512
Uploaded PennFudanPedData/PennFudanPed/Annotation/FudanPed00020.txt, 113 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/Annotation/PennPed00066.txt
Uploading PennFudanPedData/PennFudanPed/Annotation/PennPed00067.txt
Uploaded PennFudanPedData/PennFudanPed/Annotation/PennPed00047.txt, 114 files out of an estimated total of 512
Uploaded PennFudanPedData/PennFudanPed/Annotation/FudanPed00

Uploaded PennFudanPedData/PennFudanPed/Annotation/PennPed00094.txt, 161 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PNGImages/FudanPed00021.png
Uploaded PennFudanPedData/PennFudanPed/Annotation/FudanPed00029.txt, 162 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PNGImages/FudanPed00022.png
Uploaded PennFudanPedData/PennFudanPed/Annotation/PennPed00093.txt, 163 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PNGImages/FudanPed00023.png
Uploaded PennFudanPedData/PennFudanPed/Annotation/PennPed00092.txt, 164 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PNGImages/FudanPed00024.png
Uploaded PennFudanPedData/PennFudanPed/Annotation/PennPed00085.txt, 165 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PNGImages/FudanPed00025.png
Uploaded PennFudanPedData/PennFudanPed/Annotation/PennPed00090.txt, 166 files out of an estimated total 

WARNING - Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))': /azureml-blobstore-e47496c6-9688-4277-a05b-ceb722514b9d/PennFudanPedData/PNGImages/FudanPed00015.png


...

Uploaded PennFudanPedData/PennFudanPed/PedMasks/FudanPed00021_mask.png, 333 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/FudanPed00023_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/FudanPed00022_mask.png, 334 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/FudanPed00024_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/FudanPed00023_mask.png, 335 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/FudanPed00025_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/FudanPed00024_mask.png, 336 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/FudanPed00026_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/FudanPed00025_mask.png, 337 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/FudanPed00027_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/FudanPed00026_mas

Uploaded PennFudanPedData/PennFudanPed/PedMasks/FudanPed00070_mask.png, 384 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/FudanPed00074_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/FudanPed00071_mask.png, 385 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/PennPed00001_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/FudanPed00072_mask.png, 386 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/PennPed00002_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/FudanPed00047_mask.png, 387 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/PennPed00003_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/FudanPed00050_mask.png, 388 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/PennPed00004_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/FudanPed00073_mask.pn

Uploaded PennFudanPedData/PennFudanPed/PedMasks/PennPed00060_mask.png, 452 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/PennPed00068_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/PennPed00052_mask.png, 453 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/PennPed00069_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/PennPed00061_mask.png, 454 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/PennPed00070_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/PennPed00062_mask.png, 455 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/PennPed00071_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/PennPed00063_mask.png, 456 files out of an estimated total of 512
Uploading PennFudanPedData/PennFudanPed/PedMasks/PennPed00072_mask.png
Uploaded PennFudanPedData/PennFudanPed/PedMasks/PennPed00064_mask.png, 457 

Uploaded PennFudanPedData/PennFudanPed/PNGImages/PennPed00007.png, 507 files out of an estimated total of 512
Uploaded PennFudanPedData/PennFudanPed/PNGImages/PennPed00019.png, 508 files out of an estimated total of 512
Uploaded PennFudanPedData/PennFudanPed/PNGImages/PennPed00043.png, 509 files out of an estimated total of 512
Uploaded PennFudanPedData/PennFudanPed/PNGImages/FudanPed00046.png, 510 files out of an estimated total of 512
Uploaded PennFudanPedData/PennFudanPed/PNGImages/PennPed00005.png, 511 files out of an estimated total of 512
Uploaded PennFudanPedData/PennFudanPed/PNGImages/FudanPed00026.png, 512 files out of an estimated total of 512
Uploaded 512 files


$AZUREML_DATAREFERENCE_463ce198fca04e1cb749d5415cdbde44

### Register a dataset


In [24]:
from azureml.core import Dataset

dataset_name = 'penn_ds'
datastore_paths = [(ds, ds_path)]
penn_ds = Dataset.File.from_files(path=datastore_paths)
penn_ds.register(workspace=ws,
                 name=dataset_name,
                 description='Penn Fudan pedestrian data',
                 create_new_version=True)

{
  "source": [
    "('workspaceblobstore', 'PennFudanPedData')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "name": "penn_ds",
    "version": 5,
    "description": "Penn Fudan pedestrian data",
    "workspace": "Workspace.create(name='gopalv-ws', subscription_id='15ae9cb6-95c1-483d-a0e3-b1a1a3b06324', resource_group='aifxdemo')"
  }
}

### Create an experiment

In [ ]:
from azureml.core import Experiment

experiment_name = 'pytorch-peds'
experiment = Experiment(ws, name=experiment_name)

### Specify dependencies with a custom Dockerfile

There are a number of ways to [use environments](https://docs.microsoft.com/azure/machine-learning/how-to-use-environments) for specifying dependencies during model training. In this case, we use a custom Dockerfile.

In [ ]:
from azureml.core import Environment

my_env = Environment(name='maskr-docker')
my_env.docker.enabled = True
with open("dockerfiles/Dockerfile1", "r") as f:
    dockerfile_contents=f.read()
my_env.docker.base_dockerfile=dockerfile_contents
my_env.docker.base_image = None
# my_env.docker.gpu_support = True
my_env.python.interpreter_path = '/opt/miniconda/bin/python'
my_env.python.user_managed_dependencies = True



### Create a ScriptRunConfig

Use the [ScriptRunConfig](https://docs.microsoft.com/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py) class to define your run. Specify the source driectory, compute target, and environment.

In [ ]:
from azureml.train.dnn import PyTorch
from azureml.core import ScriptRunConfig

model_name = 'pytorch-peds'
output_dir = './outputs/'
n_epochs = 10

script_args = [
    '--dataset_name', dataset_name,
    '--model_name', model_name,
    '--output_dir', output_dir,
    '--n_epochs', n_epochs,
    '--ds_path', ds_path
]
# Add training script to run config
runconfig = ScriptRunConfig(
    source_directory=project_folder,
    script="script.py",
    arguments=script_args)

# Attach compute target to run config
runconfig.run_config.target = cluster_name

# Uncomment the line below if you want to try this locally first
#runconfig.run_config.target = "local"

# Attach environment to run config
runconfig.run_config.environment = my_env

### Submit your run

In [ ]:
# Submit run 
run = experiment.submit(runconfig)

# to get more details of your run
print(run.get_details())

### Monitor your run

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

## Test your model

Now that we are done training, let's see how well this model actually performs.

### Get your latest run
First, pull the latest run using `experiment.get_runs()`, which lists runs from `experiment` in reverse chronological order.

In [ ]:
from azureml.core import Run

last_successful = next(experiment.get_runs())

In [ ]:
last_successful.get_properties()

### Register your model
Next, register the model from your run.

In [ ]:
import os

last_successful.get_file_names()
last_successful.register_model(model_name=model_name, model_path=os.path.join(output_dir, model_name))

### Download your model
Next, download this registered model. Notice how we can initialize the `Model` object with the name of the registered model, rather than a path to the file itself.

In [ ]:
from azureml.core import Model

model = Model(workspace=ws, name=model_name)
path = model.download(target_dir=project_folder, exist_ok=True)

### Use your model to make a prediction

Run inferencing on a single test image and display the results.

In [ ]:
!pip install torchvision==0.5.0
!conda install cudatoolkit=10.1 -c pytorch -y

In [ ]:
import torch
from azureml.core import Dataset
from data import PennFudanDataset
from script import get_transform

from model import get_instance_segmentation_model
from script import NUM_CLASSES

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Instantiate model with correct weights, cast to correct device, place in evaluation mode
predict_model = get_instance_segmentation_model(NUM_CLASSES)
predict_model.to(device)
predict_model.load_state_dict(torch.load(path, map_location=device))
predict_model.eval()

# Load dataset
penn_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)
dataset_test = PennFudanDataset(
    dataset=penn_ds,
    transforms=get_transform(train=False),
    target_dir='local-data')


# pick one image from the test set
img, _ = dataset_test[0]

with torch.no_grad():
    prediction = predict_model([img.to(device)])

# model = torch.load(path)
#torch.load(model.get_model_path(model_name='outputs/model.pt'))

In [6]:
from data import PennFudanDataset
from azureml.core import Dataset
from script import get_transform


dataset_name = 'penn_ds'
penn_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)
dataset_test = PennFudanDataset(
    dataset=penn_ds,
    transforms=get_transform(train=False),
    target_dir='local-data')

FileNotFoundError: [Errno 2] No such file or directory: 'local-data/PNGImages'

In [12]:
penn_ds.to_path()

array(['/PennFudanPedData/Annotation/FudanPed00001.txt',
       '/PennFudanPedData/Annotation/FudanPed00002.txt',
       '/PennFudanPedData/Annotation/FudanPed00003.txt',
       '/PennFudanPedData/Annotation/FudanPed00004.txt',
       '/PennFudanPedData/Annotation/FudanPed00005.txt',
       '/PennFudanPedData/Annotation/FudanPed00006.txt',
       '/PennFudanPedData/Annotation/FudanPed00007.txt',
       '/PennFudanPedData/Annotation/FudanPed00008.txt',
       '/PennFudanPedData/Annotation/FudanPed00009.txt',
       '/PennFudanPedData/Annotation/FudanPed00010.txt',
       '/PennFudanPedData/Annotation/FudanPed00011.txt',
       '/PennFudanPedData/Annotation/FudanPed00012.txt',
       '/PennFudanPedData/Annotation/FudanPed00013.txt',
       '/PennFudanPedData/Annotation/FudanPed00014.txt',
       '/PennFudanPedData/Annotation/FudanPed00015.txt',
       '/PennFudanPedData/Annotation/FudanPed00016.txt',
       '/PennFudanPedData/Annotation/FudanPed00017.txt',
       '/PennFudanPedData/Annot

### Display the input image

In [ ]:
Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

### Display the predicted mask

In [ ]:
Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())

In [ ]:
import os
import sys
py_dll_path = os.path.join(sys.exec_prefix, 'Library', 'bin')
os.add_dll_directory(py_dll_path)

In [ ]:
penn_ds.to_path()